## 행위 패턴 개요

행위 패턴은 이름 그대로 객체의 역할에 초점을 둔다. 더 큰 기능을 구현하기 위한 객체 간의 상호 작용을 중요시한다. 행동 패턴에서 객체는 상호작용하지만 느슨하게 결합돼 있다. 

## 옵서버 패턴 이해

옵서버 패턴에서 객체(서브젝트)는 자식(옵서버)의 목록을 유지하며 서브젝트가 옵서버에 정의된 메소드를 호출할 때마다 옵서버에 이를 알린다. 분산형 애플리케이션에는 사용자가 요청한 작업을 수행하는 다수의 서비스가 엮여있다. 각 서비스가 수행하는 다양한 연산은 객체 상태에 직접적인 영향을 받는다.

브로드캐스트나 게시/구독(pub/sub) 시스템에서 옵서버 디자인 패턴이 자주 사용된다. 블로그를 예로 들면, 특정 블로그를 구독하고 새로운 글이 추가됐을 때 알람을 받는 경우가 있을 때 이 상황을 옵서버 패턴으로 구현한다면 블로그는 구독자 또는 옵서버의 목록을 유지하는 서브젝트다. 블로그에 새로운 글이 등록되면 이메일 또는 각 옵서버가 선택한 방식으로 알림을 보낸다.

옵서버 패턴의 목적은 다음과 같다.

* 객체 간 일대다 관계를 형성하고 객체의 상태를 다른 종속 객체에 자동으로 알린다.
* 서브젝트의 핵심 부분을 캡슐화한다.

옵서버 패턴은 다음과 같은 상황에 적합하다.

* 분산 시스템의 이벤트 서비스를 구현할 때
* 뉴스 에이전시 프레임워크
* 주식 시장 모델

파이썬으로 구현한 옵서버 디자인 패턴

In [1]:
class Subject:
    def __init__(self):
        self.__observers = []
    
    def register(self, observer):
        self.__observers.append(observer)
        
    def notifyAll(self, *args, **kwargs):
        for observer in self.__observers:
            observer.notify(self, *args, **kwargs)

class Observer1:
    def __init__(self, subject):
        subject.register(self)
    
    def notify(self, subject, *args):
        print(type(self).__name__,':: Got', args, 'From', subject)


class Observer2:
    def __init__(self, subject):
        subject.register(self)
    
    def notify(self, subject, *args):
        print(type(self).__name__,':: Got', args, 'From', subject)

subject = Subject()
observer1 = Observer1(subject)
observer2 = Observer2(subject)
subject.notifyAll('notification')

Observer1 :: Got ('notification',) From <__main__.Subject object at 0x10544ec18>
Observer2 :: Got ('notification',) From <__main__.Subject object at 0x10544ec18>


옵서버 패턴의 구성원의 각 역할은 다음과 같다.

* 서브젝트: Subject는 Observer를 관리한다. Observer는 Subject 클래스의 register()와 deregister() 메소드를 호출해 자신을 등록한다.
* 옵서버: 서브젝트를 감시하는 객체를 위한 인터페이스를 제공한다.
* ConcreteObserver: Subject의 상태를 저장한다. 서브젝트에 대한 정보와 실제 상태를 일관되게 유지하기 위해 Observer 인터페이스를 구현한다.

ConcreteObserver는 Observer 인터페이스를 구현해 자신을 Subject에 등록한다. 상태 변화가 있을 때마다 Subject는 Observer의 알림 메소드를 통해 모든 ConcreteObserver에 알린다.

## 옵서버 패턴 사용 사례


뉴스 에이전시는 일반적으로 여러 곳에서 뉴스를 모아 구독자에게 전달한다. 이 경우의 디자인 요소를 생각해보자.

뉴스 게시자인 서브젝트부터 구현한다.

* 서브젝트의 행동은 NewsPublisher 클래스에 구현한다.
* NewsPublisher는 구독자가 구현할 인터페이스를 제공한다.
* Observer는 attach 메소드를 통해 자신을 NewsPublisher에 등록하고 detach 메소드로 등록을 취소한다.
* subscrobers는 Subject에 등록된 구독자 목록을 반환한다.
* notifySubcriber는 NewsPublisher에 등록된 모든 구독자에게 알림을 보낸다.
* 뉴스 게시자는 addNews() 메소드로 새로운 뉴스를 등록하고 getNews()로 최신 뉴스를 확인한 뒤 Observer에 전달한다.

NewsPublisher

In [2]:
class NewsPublisher:
    def __init__(self):
        self.__subscribers = []
        self.__latestNews = None
    
    def attach(self, subscriber):
        self.__subscribers.append(subscriber)
    
    def detach(self):
        return self.__subscribers.pop()
    
    def subscribers(self):
        return [type(x).__name__ for x in self.__subscribers]
    
    def notifySubscribers(self):
        for sub in self.__subscribers:
            sub.update()
    
    def addNews(self, news):
        self.__latestNews = news
    
    def getNews(self):
        return "Got News", self.__latestNews

Observer 인터페이스

* Subscriber는 Observer를 나타낸다. 모든 ConcreteObserver의 추상 기본 클래스다.
* Subscriber에는 ConcreteObserver가 구현해야 하는 update 메소드가 있다.
* ConcreteObserver는 update()를 구현해 Subject(NewsPublisher)로부터 새로운 뉴스 알림을 받는다.

Subscriber 추상 클래스 구현

In [3]:
from abc import ABCMeta, abstractmethod

class Subscriber(metaclass=ABCMeta):
    @abstractmethod
    def update(self):
        pass

ConcreteObserver 클래스의 역할

* EmailSubscriber와 SMSSubscriber는 Subscriber 인터페이스를 구현하는 옵서버다.
* AnyOtherObserver는 Observer와 Subject의 느슨한 관계를 나타내는 또 다른 옵서버다.
* 각 ConcreteObserver의 \__init__() 메소드는 attach 메소드를 통해 자신을 NewsPublisher에 등록
* NewsPublisher는 내부적으로 ConcreteObserver의 update() 메소드를 호출해 새로운 뉴스를 알린다.

SMSSubscriber 클래스 구현

In [6]:
class SMSSubscriber:
    def __init__(self, publisher):
        self.publisher = publisher
        self.publisher.attach(self)
    
    def update(self):
        print(type(self).__name__, self.publisher.getNews())

class EmailSubscriber:
    def __init__(self, publisher):
        self.publisher = publisher
        self.publisher.attach(self)
    
    def update(self):
        print(type(self).__name__, self.publisher.getNews())

class AnyOtherSubscriber:
    def __init__(self, publisher):
        self.publisher = publisher
        self.publisher.attach(self)
    
    def update(self):
        print(type(self).__name__, self.publisher.getNews())

In [7]:
news_publisher = NewsPublisher()
for Subscriber in [SMSSubscriber, EmailSubscriber, AnyOtherSubscriber]:
    Subscriber(news_publisher)
print("\n Subscriber:", news_publisher.subscribers())

news_publisher.addNews("Hello")
news_publisher.notifySubscribers()

print("\nDetached: ", type(news_publisher.detach()).__name__)
print("\nSubscribers:", news_publisher.subscribers())

news_publisher.addNews("Next News")
news_publisher.notifySubscribers()


 Subscriber: ['SMSSubscriber', 'EmailSubscriber', 'AnyOtherSubscriber']
SMSSubscriber ('Got News', 'Hello')
EmailSubscriber ('Got News', 'Hello')
AnyOtherSubscriber ('Got News', 'Hello')

Detached:  AnyOtherSubscriber

Subscribers: ['SMSSubscriber', 'EmailSubscriber']
SMSSubscriber ('Got News', 'Next News')
EmailSubscriber ('Got News', 'Next News')


## 옵서버 패턴 메소드

Subject의 변경 사항을 Observer에 알리는 방법에는 푸시와 풀 두 가지 모델이 있다.

### 풀 모델

* Subject는 변경 사항이 있음을 등록된 Observer에 브로드캐스트한다.
* Observer는 직접 게시자에게 변경 사항을 요청하고 끌어와야 한다.
* 풀 모델은 Subject가 Observer에 알리는 단계와 Observer가 Subject로부터 필요한 데이터를 받아오는 두 단계가 필요하므로 비효율적이다.

### 푸시 모델

* 풀 모델과 달리 Subject가 Observer에 데이터를 보낸다.
* Subject는 Observer가 필요로 하지 않는 데이터까지 보낼 수 있다. 따라서 쓸데없이 많은 양의 데이터를 전송해 응답 시간이 늦어질 수 있다.
* 성능을 위해 Subject는 필요한 데이터만 보내야 한다.

## 느슨한 결합과 옵서버 패턴

느슨한 결합이란 객체가 상호 작용하는 다른 객체에 대해 알고 있는 정도를 의미한다.

느슨한 결합을 추구한 설계는 객체 간의 의존도를 줄여 유연한 객체지향 시스템을 만들 수 있다.

느슨한 결합의 효과
    
    * 한 부분에 대한 수정이 얘기치 않게 다른 부분까지 영향을 끼치는 위험을 줄인다.
    * 테스트와 유지 보수 및 장애 처리가 쉽다.
    * 시스템을 쉽게 여러 부분으로 분리할 수 있다.
    
옵서버 패턴은 Subject와 Observer의 느슨한 결합을 추구한다.

    * Subject는 정확히 Observer가 어떤 인터페이스를 구현하는지 모른다. ConcreteObserver도 모른다.
    * 언제든지 새로운 Observer를 추가할 수 있다.
    * 새로운 Observer를 추가해도 Subject를 수정할 필요가 없다. 
    * Subject와 Observer는 독립적이다.

## 옵서버 패턴의 장단점

장점
    
    * 객체 간의 느슨한 결합 원칙
    * Subject, Observer 클래스 수정없이 객체 간 자유롭게 데이터를 주고받을 수 있다.
    * 새로운 Observer를 언제든지 추가/제거할 수 있다.
   
단점

    * ConcreteObserver는 상속을 통해 Observer 인터페이스를 구현한다. 컴포지션에 대한 선택권이 없다.
    * 제대로 구현되지 않은 Observer 클래스를 복잡도를 높이고 성능 저하의 원인이 될 수 있다.
    * 애플리케이션에서 알림 기능은 간혹 신뢰할 수 없으며 레이스 상태 또는 비일관성을 초래할 수 있다.